<a href="https://colab.research.google.com/github/benriworks/Topic-Translate-Feed/blob/main/%E5%AE%8C%E6%88%90%E7%89%8820251130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google ドライブをマウントしています...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
出力ディレクトリ: /content/drive/MyDrive/gemini_batch_outputs
作成したリクエストファイル: /content/drive/MyDrive/gemini_batch_outputs/my-batch-image-requests.jsonl
Uploaded file: files/vihwmjk0ks0o
Created batch job: batches/qqg22u9frne2etdg5m5nvsn5b07ooiearrq1
Current state: JOB_STATE_PENDING
Current state: JOB_STATE_RUNNING
Current state: JOB_STATE_RUNNING
Job finished with state: JOB_STATE_SUCCEEDED
結果ファイルを保存しました: /content/drive/MyDrive/gemini_batch_outputs/batch-qqg22u9frne2etdg5m5nvsn5b07ooiearrq1
画像を保存しました: /content/drive/MyDrive/gemini_batch_outputs/request-1_image_1.png
画像を保存しました: /content/drive/MyDrive/gemini_batch_outputs/request-2_image_2.png


In [2]:
import json
import time
import base64
from pathlib import Path
from io import BytesIO

from google.colab import drive
from google import genai
from google.genai import types
from PIL import Image
import google.colab.userdata
import pandas as pd  # ★ 追加：Excel用

# Google Drive マウント
DRIVE_MOUNT_PATH = "/content/drive"
DRIVE_ROOT = Path(DRIVE_MOUNT_PATH) / "MyDrive"
OUTPUT_DIR = DRIVE_ROOT / "gemini_batch_outputs"

print("Google ドライブをマウントしています...")
drive.mount(DRIVE_MOUNT_PATH)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"出力ルートディレクトリ: {OUTPUT_DIR}")

# === 日時付きサブフォルダを作成 ===
timestamp = time.strftime("%Y%m%d_%H%M%S")  # 例: 20251130_153045
SESSION_DIR = OUTPUT_DIR / f"batch_{timestamp}"
SESSION_DIR.mkdir(parents=True, exist_ok=True)
print(f"このバッチの出力フォルダ: {SESSION_DIR}")

client = genai.Client(api_key=google.colab.userdata.get('GOOGLE_API_KEY'))

# === Excel からプロンプト群を読み込み ===
excel_path = DRIVE_ROOT / "prompt.xlsx"  # 例: /content/drive/MyDrive/prompt.xlsx
print(f"Excel を読み込みます: {excel_path}")
df = pd.read_excel(excel_path)

# 想定カラム: id, prompt, aspect_ratio, image_size
# 足りない/NaN の場合はデフォルト値で補完
DEFAULT_ASPECT = "9:16"
DEFAULT_SIZE = "1K"

# === 1. JSONLリクエストファイル作成 ===
file_name = SESSION_DIR / "my-batch-image-requests.jsonl"
requests = []

for idx, row in df.iterrows():
    # id は key として使う（数値の場合もあるので str()）
    key = str(row["id"])
    prompt_text = str(row["prompt"])

    # aspect_ratio, image_size は空ならデフォルトにする
    aspect_ratio = str(row["aspect_ratio"]) if pd.notna(row["aspect_ratio"]) else DEFAULT_ASPECT
    image_size = str(row["image_size"])    if pd.notna(row["image_size"])    else DEFAULT_SIZE

    req = {
        "key": key,
        "request": {
            "contents": [
                {
                    "parts": [
                        {"text": prompt_text}
                    ]
                }
            ],
            # ★ JSONL では camelCase の "responseModalities"
            "generation_config": {
                "responseModalities": ["TEXT", "IMAGE"],
                "image_config": {
                    "aspect_ratio": aspect_ratio,
                    "image_size": image_size
                }
            }
        }
    }
    requests.append(req)

# JSONLファイルとして保存（1行 = 1リクエスト）
with open(file_name, "w", encoding="utf-8") as f:
    for req in requests:
        f.write(json.dumps(req, ensure_ascii=False) + "\n")

print(f"作成したリクエストファイル: {file_name}")
print(f"行数（リクエスト数）: {len(requests)}")

# === 2. ファイルアップロード ===
uploaded_file = client.files.upload(
    file=str(file_name),
    config=types.UploadFileConfig(
        display_name="my-batch-image-requests",
        mime_type="jsonl"
    )
)
print(f"Uploaded file: {uploaded_file.name}")

# === 3. バッチジョブ作成 ===
file_batch_job = client.batches.create(
    model="gemini-3-pro-image-preview",
    src=uploaded_file.name,
    config={"display_name": "file-image-upload-job-1"},
)
print(f"Created batch job: {file_batch_job.name}")

# === 4. ステータス監視 ===
job_name = file_batch_job.name
completed_states = {'JOB_STATE_SUCCEEDED', 'JOB_STATE_FAILED', 'JOB_STATE_CANCELLED', 'JOB_STATE_EXPIRED'}

batch_job = client.batches.get(name=job_name)
while batch_job.state.name not in completed_states:
    print(f"Current state: {batch_job.state.name}")
    time.sleep(10)
    batch_job = client.batches.get(name=job_name)

print(f"Job finished with state: {batch_job.state.name}")

# === 5. 結果のダウンロード・保存 ===
if batch_job.state.name == "JOB_STATE_SUCCEEDED":
    result_file_name = batch_job.dest.file_name
    # 結果ファイルもこのバッチ用フォルダに保存
    result_path = SESSION_DIR / Path(result_file_name).name
    file_content_bytes = client.files.download(file=result_file_name)
    result_path.write_bytes(file_content_bytes)
    print(f"結果ファイルを保存しました: {result_path}")

    # テキスト結果もこのフォルダへ
    text_output_path = SESSION_DIR / "batch_text_results.txt"
    text_lines = []
    image_counter = 0

    for line in result_path.read_text().splitlines():
        if not line:
            continue
        parsed_response = json.loads(line)
        key = parsed_response.get("key", "response")

        if "response" in parsed_response and parsed_response["response"]:
            parts = parsed_response["response"]["candidates"][0]["content"]["parts"]
            for part in parts:
                if part.get("text"):
                    text_lines.append(f"{key}: {part['text']}")
                elif part.get("inlineData"):
                    image_counter += 1
                    data = base64.b64decode(part["inlineData"]["data"])
                    image = Image.open(BytesIO(data))
                    # 画像もこのバッチフォルダに保存
                    image_path = SESSION_DIR / f"{key}_image_{image_counter}.png"
                    image.save(image_path)
                    print(f"画像を保存しました: {image_path}")
        elif "error" in parsed_response:
            text_lines.append(f"{key}: Error - {parsed_response['error']}")

    if text_lines:
        text_output_path.write_text("\n".join(text_lines), encoding="utf-8")
        print(f"テキスト結果を保存しました: {text_output_path}")
elif batch_job.state.name == "JOB_STATE_FAILED":
    print(f"Error: {batch_job.error}")


Google ドライブをマウントしています...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
出力ルートディレクトリ: /content/drive/MyDrive/gemini_batch_outputs
このバッチの出力フォルダ: /content/drive/MyDrive/gemini_batch_outputs/batch_20251130_120134
Excel を読み込みます: /content/drive/MyDrive/prompt.xlsx
作成したリクエストファイル: /content/drive/MyDrive/gemini_batch_outputs/batch_20251130_120134/my-batch-image-requests.jsonl
行数（リクエスト数）: 2
Uploaded file: files/nuqa860a5sk3
Created batch job: batches/6cgnsocq14fnfvgcf8c6ejvgyd3xovdizgah
Current state: JOB_STATE_PENDING
Current state: JOB_STATE_PENDING
Current state: JOB_STATE_RUNNING
Current state: JOB_STATE_RUNNING
Current state: JOB_STATE_RUNNING
Job finished with state: JOB_STATE_SUCCEEDED
結果ファイルを保存しました: /content/drive/MyDrive/gemini_batch_outputs/batch_20251130_120134/batch-6cgnsocq14fnfvgcf8c6ejvgyd3xovdizgah
画像を保存しました: /content/drive/MyDrive/gemini_batch_outputs/batch_20251130_120134/img-001_image_1.png
画像を保存